# Part B: Sentence Segmentation in Urdu

## Approach
Urdu sentence segmentation is challenging because:
- Urdu uses its own punctuation (e.g., `۔` full stop, `؟` question mark, `!` exclamation)
- Spaces may be omitted between words (Space Omission Problem)
- Extra spaces may be inserted inside words (Space Insertion Problem)
- Multiple sentences are often run together without proper delimiters in noisy corpora

**Strategy:**
1. Normalize whitespace (fix Space Insertion Problem)
2. Identify sentence boundaries using Urdu punctuation markers (`۔`, `؟`, `!`, `؟`) and contextual end-word cues
3. Split on detected boundaries
4. Evaluate using Precision, Recall, and F1-score against a manually annotated gold standard

In [1]:
# ============================================================
# Part B: Urdu Sentence Segmentation
# ============================================================

import re
from typing import List, Tuple


# ---------------------------------------------------------------
# Step 1: Text Normalization
# Fix Space Insertion Problem: remove spurious spaces inside Urdu words
# Fix Space Omission Problem: ensure space after sentence-ending punctuation
# ---------------------------------------------------------------

# Common Urdu sentence-ending words (end-of-sentence cues)
# These are common Urdu words that typically appear at the end of a sentence
URDU_SENTENCE_FINAL_WORDS = [
    'ہے', 'ہیں', 'تھا', 'تھی', 'تھے', 'گا', 'گی', 'گے',
    'دیا', 'لیا', 'کیا', 'آیا', 'گیا', 'ہوا', 'ہوئی', 'ہوئے',
    'چاہیے', 'سکتا', 'سکتی', 'سکتے', 'چاہتا', 'چاہتی',
    'پہنچا', 'پہنچی', 'نہیں', 'ملا', 'ملی', 'رہا', 'رہی', 'رہے'
]

# Urdu punctuation that marks sentence end
URDU_END_PUNCT = r'[۔؟!]'


def normalize_whitespace(text: str) -> str:
    """
    Fix Space Insertion Problem:
    Collapse multiple consecutive spaces into a single space.
    Also strip leading/trailing whitespace from the full text.
    """
    # Replace multiple spaces/tabs with a single space
    text = re.sub(r'[ \t]+', ' ', text)
    # Normalize newlines
    text = re.sub(r'\n+', '\n', text)
    return text.strip()


def ensure_space_after_punct(text: str) -> str:
    """
    Ensure there is at least one space after Urdu sentence-ending punctuation
    (helps with Space Omission Problem at sentence boundaries).
    """
    # Add space after ۔ ؟ ! if not followed by space or newline
    text = re.sub(r'([۔؟!])([^\s])', r'\1 \2', text)
    return text


def preprocess(text: str) -> str:
    """Full preprocessing pipeline."""
    text = normalize_whitespace(text)
    text = ensure_space_after_punct(text)
    return text


# ---------------------------------------------------------------
# Step 2: Sentence Segmentation
# ---------------------------------------------------------------

def segment_sentences(text: str) -> List[str]:
    """
    Segment Urdu text into individual sentences.

    Strategy:
    1. Preprocess (normalize whitespace, ensure space after punctuation)
    2. Split on Urdu sentence-ending punctuation (۔ ؟ !)
    3. Apply end-word heuristic: if a token matches a known sentence-final
       word and the next token starts a new potential sentence (starts with
       a capital letter equivalent or a recognized sentence-starting pattern),
       insert a boundary.
    4. Clean up empty segments.
    """
    text = preprocess(text)

    # Primary split: on Urdu end punctuation, keeping the punctuation
    # Use a lookahead to split AFTER the punctuation character
    raw_segments = re.split(r'(?<=[۔؟!])\s*', text)

    sentences = []
    buffer = ''

    for seg in raw_segments:
        seg = seg.strip()
        if not seg:
            continue

        # Check if this segment already ends with Urdu punctuation
        if re.search(URDU_END_PUNCT + r'$', seg):
            if buffer:
                sentences.append((buffer + ' ' + seg).strip())
                buffer = ''
            else:
                sentences.append(seg)
        else:
            # No ending punctuation: apply end-word heuristic
            words = seg.split()
            sub_sentence = []
            for i, word in enumerate(words):
                sub_sentence.append(word)
                # If word matches a sentence-final cue and there are more words
                if word in URDU_SENTENCE_FINAL_WORDS and i < len(words) - 1:
                    candidate = ' '.join(sub_sentence).strip()
                    if buffer:
                        sentences.append((buffer + ' ' + candidate).strip())
                        buffer = ''
                    else:
                        sentences.append(candidate)
                    sub_sentence = []
            # Remaining words go to buffer
            remainder = ' '.join(sub_sentence).strip()
            if remainder:
                buffer = (buffer + ' ' + remainder).strip() if buffer else remainder

    # Flush remaining buffer
    if buffer:
        sentences.append(buffer)

    # Final cleanup: remove empty strings and strip whitespace
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences


# ---------------------------------------------------------------
# Step 3: Evaluation Function
# ---------------------------------------------------------------

def evaluate_segmentation(
    predicted: List[str],
    gold: List[str]
) -> dict:
    """
    Evaluate sentence segmentation using boundary-based Precision, Recall, F1.

    Method:
    - Reconstruct the original text from both predicted and gold sentence lists.
    - Identify boundary positions (character offsets where each sentence ends)
      in the original text.
    - Compare predicted boundaries to gold boundaries.

    Returns:
        dict with keys: precision, recall, f1, correct, predicted_count, gold_count
    """
    def get_boundaries(sentences: List[str]) -> set:
        """Return set of cumulative character offsets at each sentence boundary."""
        boundaries = set()
        offset = 0
        for i, sent in enumerate(sentences[:-1]):  # no boundary after last sentence
            offset += len(sent)
            boundaries.add(offset)
            offset += 1  # account for space between sentences
        return boundaries

    pred_boundaries = get_boundaries(predicted)
    gold_boundaries = get_boundaries(gold)

    # Allow a small tolerance window (±2 chars) for boundary matching
    TOLERANCE = 2

    correct = 0
    matched_gold = set()
    for pb in pred_boundaries:
        for gb in gold_boundaries:
            if abs(pb - gb) <= TOLERANCE and gb not in matched_gold:
                correct += 1
                matched_gold.add(gb)
                break

    precision = correct / len(pred_boundaries) if pred_boundaries else 0.0
    recall = correct / len(gold_boundaries) if gold_boundaries else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    return {
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'f1': round(f1, 4),
        'correct_boundaries': correct,
        'predicted_boundaries': len(pred_boundaries),
        'gold_boundaries': len(gold_boundaries)
    }


print("Part B functions defined successfully.")

Part B functions defined successfully.


In [2]:
# ---------------------------------------------------------------
# Test on sample Urdu text (from the assignment PDF)
# ---------------------------------------------------------------

# Example from assignment: two Urdu sentences joined together
sample_text = (
    "بے چاری عوام چونکہ ہمیشہ سے دھوکہ کھانے کی عادی رہی ہے اس لئے تبدیل سرکار کی چکنی چپڑی باتوں میں آگئی "
    "اور اپنے بہتر مستقبل کے لئے نئی حکومت کو اقتدار کے ایوانوں تک پہنچا دیا۔"
)

# Multi-sentence Urdu test corpus
multi_sentence_text = (
    "پاکستان ایک خوبصورت ملک ہے۔ اس کی آبادی بیس کروڑ سے زیادہ ہے۔ "
    "اسلام آباد پاکستان کا دارالحکومت ہے۔ یہاں بہت سے تاریخی مقامات موجود ہیں۔ "
    "کیا آپ کبھی پاکستان گئے ہیں؟ پاکستانی کھانا بہت لذیذ ہوتا ہے!"
)

# Text with space insertion problem
space_insertion_text = (
    "پاکستان  ایک   خوبصورت   ملک  ہے۔  اس   کی  آبادی  بہت  زیادہ   ہے۔"
)

print("=" * 60)
print("TEST 1: Assignment sample text")
print("=" * 60)
print("INPUT:\n", sample_text)
segs = segment_sentences(sample_text)
print(f"\nSEGMENTED ({len(segs)} sentence(s)):")
for i, s in enumerate(segs, 1):
    print(f"  [{i}] {s}")

print()
print("=" * 60)
print("TEST 2: Multi-sentence Urdu corpus")
print("=" * 60)
print("INPUT:\n", multi_sentence_text)
segs2 = segment_sentences(multi_sentence_text)
print(f"\nSEGMENTED ({len(segs2)} sentence(s)):")
for i, s in enumerate(segs2, 1):
    print(f"  [{i}] {s}")

print()
print("=" * 60)
print("TEST 3: Space insertion problem text")
print("=" * 60)
print("INPUT:\n", space_insertion_text)
segs3 = segment_sentences(space_insertion_text)
print(f"\nSEGMENTED ({len(segs3)} sentence(s)):")
for i, s in enumerate(segs3, 1):
    print(f"  [{i}] {s}")

TEST 1: Assignment sample text
INPUT:
 بے چاری عوام چونکہ ہمیشہ سے دھوکہ کھانے کی عادی رہی ہے اس لئے تبدیل سرکار کی چکنی چپڑی باتوں میں آگئی اور اپنے بہتر مستقبل کے لئے نئی حکومت کو اقتدار کے ایوانوں تک پہنچا دیا۔

SEGMENTED (1 sentence(s)):
  [1] بے چاری عوام چونکہ ہمیشہ سے دھوکہ کھانے کی عادی رہی ہے اس لئے تبدیل سرکار کی چکنی چپڑی باتوں میں آگئی اور اپنے بہتر مستقبل کے لئے نئی حکومت کو اقتدار کے ایوانوں تک پہنچا دیا۔

TEST 2: Multi-sentence Urdu corpus
INPUT:
 پاکستان ایک خوبصورت ملک ہے۔ اس کی آبادی بیس کروڑ سے زیادہ ہے۔ اسلام آباد پاکستان کا دارالحکومت ہے۔ یہاں بہت سے تاریخی مقامات موجود ہیں۔ کیا آپ کبھی پاکستان گئے ہیں؟ پاکستانی کھانا بہت لذیذ ہوتا ہے!

SEGMENTED (6 sentence(s)):
  [1] پاکستان ایک خوبصورت ملک ہے۔
  [2] اس کی آبادی بیس کروڑ سے زیادہ ہے۔
  [3] اسلام آباد پاکستان کا دارالحکومت ہے۔
  [4] یہاں بہت سے تاریخی مقامات موجود ہیں۔
  [5] کیا آپ کبھی پاکستان گئے ہیں؟
  [6] پاکستانی کھانا بہت لذیذ ہوتا ہے!

TEST 3: Space insertion problem text
INPUT:
 پاکستان  ایک   خوبصورت   مل

In [3]:
# ---------------------------------------------------------------
# Evaluation Test
# ---------------------------------------------------------------

# Gold-standard (manually defined expected segmentation)
gold_standard = [
    "پاکستان ایک خوبصورت ملک ہے۔",
    "اس کی آبادی بیس کروڑ سے زیادہ ہے۔",
    "اسلام آباد پاکستان کا دارالحکومت ہے۔",
    "یہاں بہت سے تاریخی مقامات موجود ہیں۔",
    "کیا آپ کبھی پاکستان گئے ہیں؟",
    "پاکستانی کھانا بہت لذیذ ہوتا ہے!"
]

predicted_segmentation = segment_sentences(multi_sentence_text)

results = evaluate_segmentation(predicted_segmentation, gold_standard)

print("=" * 60)
print("EVALUATION RESULTS")
print("=" * 60)
print(f"Gold boundaries       : {results['gold_boundaries']}")
print(f"Predicted boundaries  : {results['predicted_boundaries']}")
print(f"Correctly matched     : {results['correct_boundaries']}")
print(f"Precision             : {results['precision']:.4f}")
print(f"Recall                : {results['recall']:.4f}")
print(f"F1 Score              : {results['f1']:.4f}")

print("\nPredicted sentences:")
for i, s in enumerate(predicted_segmentation, 1):
    print(f"  [{i}] {s}")

EVALUATION RESULTS
Gold boundaries       : 5
Predicted boundaries  : 5
Correctly matched     : 5
Precision             : 1.0000
Recall                : 1.0000
F1 Score              : 1.0000

Predicted sentences:
  [1] پاکستان ایک خوبصورت ملک ہے۔
  [2] اس کی آبادی بیس کروڑ سے زیادہ ہے۔
  [3] اسلام آباد پاکستان کا دارالحکومت ہے۔
  [4] یہاں بہت سے تاریخی مقامات موجود ہیں۔
  [5] کیا آپ کبھی پاکستان گئے ہیں؟
  [6] پاکستانی کھانا بہت لذیذ ہوتا ہے!
